In [41]:
import pandas as pd
import numpy as np
from scipy import stats


In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
df = pd.read_csv('/content/drive/MyDrive/layer-9/train.csv')
valid = pd.read_csv('/content/drive/MyDrive/layer-9/valid.csv')
test = pd.read_csv('/content/drive/MyDrive/layer-9/test.csv')


In [44]:
df

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.019301,0.059756,0.081375,0.057481,-0.068440,-0.165913,0.035643,-0.091138,0.021688,0.057158,...,-0.035576,0.127319,0.098128,-0.058787,0.100971,-0.047754,45,NaN,1,6
1,0.049741,0.090030,0.035118,-0.013676,-0.194317,-0.101763,0.085875,-0.081317,0.112418,0.120523,...,0.020538,0.058968,0.029803,0.111324,0.036727,0.031927,45,NaN,1,6
2,0.019212,0.087779,0.093907,-0.033738,-0.141409,-0.062881,-0.071402,-0.006599,0.020372,-0.027777,...,0.119645,-0.040861,0.000548,-0.061003,-0.042450,0.063340,45,NaN,1,6
3,0.070283,0.049040,0.042126,0.122637,-0.056964,-0.113700,0.108454,0.051336,0.086610,0.141578,...,-0.124494,-0.169225,-0.046391,0.148787,0.014616,-0.140644,45,NaN,1,6
4,0.028864,0.165634,0.016302,0.036117,-0.028871,-0.147748,0.053180,0.025071,-0.004200,-0.022183,...,-0.124862,0.044907,0.084005,-0.038450,0.084371,-0.072146,45,NaN,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28515,-0.041520,0.302638,0.004811,0.211441,-0.249326,-0.113395,-0.146776,-0.146222,-0.040344,-0.124016,...,0.290069,-0.223051,-0.053575,-0.230222,-0.451943,0.117817,39,29.0,1,6
28516,-0.056359,0.061850,0.051865,0.032107,-0.005930,-0.040313,0.117453,-0.067553,0.045004,0.035735,...,0.004518,0.035248,-0.047345,-0.006539,-0.025633,0.002474,39,29.0,1,6
28517,0.019361,0.148111,0.093434,0.011795,-0.074029,-0.077628,0.049933,-0.096354,0.138556,0.130414,...,0.014918,0.015676,-0.033608,0.068212,0.049871,-0.027607,39,29.0,1,6
28518,-0.043644,0.084073,0.074749,0.091776,-0.090782,-0.163187,-0.012568,-0.051021,0.038726,0.011009,...,-0.098420,0.113067,0.028218,0.036682,0.030056,-0.084346,39,29.0,1,6


## **Pre-processing**

In [45]:
# Count missing values in the selected label column
missing_values_label_1 = df["label_1"].isnull().sum()
missing_values_label_2 = df["label_2"].isnull().sum()
missing_values_label_3 = df["label_3"].isnull().sum()
missing_values_label_4 = df["label_4"].isnull().sum()

print(missing_values_label_1)
print(missing_values_label_2)
print(missing_values_label_3)
print(missing_values_label_4)


0
480
0
0


In [46]:
#filling missing values with mode
print(df.label_2.value_counts())
print(df.label_3.value_counts())
print(df.label_4.value_counts())




26.0    4762
25.0    2849
27.0    2846
23.0    2842
31.0    2385
24.0    1906
28.0    1899
30.0    1894
22.0    1432
29.0    1424
33.0     945
36.0     481
35.0     480
34.0     478
32.0     476
41.0     474
61.0     467
Name: label_2, dtype: int64
1    22796
0     5724
Name: label_3, dtype: int64
6     19938
2      1449
0       955
12      954
7       938
13      482
1       481
11      480
10      480
3       479
5       478
9       472
4       469
8       465
Name: label_4, dtype: int64


In [47]:
# Calculate the mean of the column
mean_value = df.label_2.mean()

# Fill missing values in the specified column with the rounded mean
# df.label_2.fillna(round(mean_value), inplace=True)


In [48]:
df_copy =df.copy()
X=df_copy.drop(columns=['label_1','label_2','label_3','label_4'])

#remove outliers
# df=df[(np.abs(stats.zscore(X)) < 3).all(axis=1)]


## **Feature Engineering**

In [49]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


## Label-1 calculation

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

df_1=df.copy()
valid_1=valid.copy()
X_train_1=df_1.drop(columns=['label_1','label_2','label_3','label_4'])
y_train_1=df_1.label_1
X_valid_1=valid_1.drop(columns=['label_1','label_2','label_3','label_4'])
y_valid_1=valid_1.label_1
X_test_1 = test.copy().drop(columns=['ID'])

# knn_1=KNeighborsClassifier()
# cross_val_score(knn_1, X_1, y_1, cv=5, scoring='f1_macro').mean()




In [51]:
discrete_features = X_train_1.dtypes == int

mi_scores = make_mi_scores(X_train_1, y_train_1, discrete_features)
mi_scores.sort_values(ascending=False)

In [52]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

In [53]:
X_train_1 = X_train_1.drop(columns=last_100_features)
X_valid_1 = X_valid_1.drop(columns=last_100_features)
X_test_1 = X_test_1.drop(columns=last_100_features)




In [54]:
model_1=SVC()
model_1.fit(X_train_1, y_train_1)

SVC()

In [55]:
y_pred_1 = model_1.predict(X_valid_1)
y_test_1= model_1.predict(X_test_1)

In [56]:
print(classification_report(y_valid_1, y_pred_1))

              precision    recall  f1-score   support

           1       1.00      0.92      0.96        13
           2       1.00      1.00      1.00         9
           3       0.85      0.92      0.88        12
           4       0.67      1.00      0.80        16
           5       1.00      0.83      0.91        18
           6       1.00      1.00      1.00         9
           7       0.84      0.94      0.89        17
           8       1.00      0.71      0.83        14
           9       1.00      0.91      0.95        11
          10       1.00      1.00      1.00         8
          11       0.79      1.00      0.88        19
          12       1.00      1.00      1.00         7
          13       1.00      0.73      0.84        11
          14       1.00      0.93      0.97        15
          15       1.00      0.88      0.94        17
          16       1.00      0.93      0.96        14
          17       1.00      0.93      0.96        14
          18       1.00    

## Label-4 calculation

In [57]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.utils import resample
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

df_4=df.copy()
valid_4=valid.copy()
X_train_4=df_4.drop(columns=['label_1','label_2','label_3','label_4'])
y_train_4=df_4.label_4
X_valid_4=valid_4.drop(columns=['label_1','label_2','label_3','label_4'])
y_valid_4=valid_4.label_4
X_test_4 = test.copy().drop(columns=['ID'])


# Separate the majority class (class 6) from other classes
majority_class = 6
minority_classes = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13]

# Create DataFrames for majority and minority classes
df_majority = df_4[df_4['label_4'] == majority_class]
df_minority = df_4[df_4['label_4'].isin(minority_classes)]

# Down-sample the majority class to match the size of minority classes
num_samples_minority = len(df_minority)
df_majority_downsampled = resample(df_majority, replace=False, n_samples=num_samples_minority, random_state=42)

# Combine minority and down-sampled majority classes
df_balanced = pd.concat([df_minority, df_majority_downsampled])

# Shuffle the balanced dataset
df_4 = df_balanced.sample(frac=1, random_state=42)

# Now df_balanced contains a balanced dataset
df_4.label_4.value_counts()

# knn_4=KNeighborsClassifier()
# cross_val_score(knn_4, X_4, y_4, cv=5, scoring='f1_macro').mean()


6     8582
2     1449
0      955
12     954
7      938
13     482
1      481
10     480
11     480
3      479
5      478
9      472
4      469
8      465
Name: label_4, dtype: int64

In [58]:
discrete_features = X_train_4.dtypes == int

mi_scores = make_mi_scores(X_train_4, y_train_4, discrete_features)
mi_scores.sort_values(ascending=False)

In [59]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

In [60]:
X_train_4 = X_train_4.drop(columns=last_100_features)
X_valid_4 = X_valid_4.drop(columns=last_100_features)
X_test_4 = X_test_4.drop(columns=last_100_features)



In [61]:
model_4=SVC()
model_4.fit(X_train_4, y_train_4)

SVC()

In [62]:
y_pred_4 = model_4.predict(X_valid_4)
y_test_4= model_4.predict(X_test_4)

In [63]:
print(classification_report(y_valid_4, y_pred_4))

              precision    recall  f1-score   support

           0       1.00      0.95      0.98        21
           1       1.00      0.82      0.90        11
           2       1.00      0.85      0.92        27
           3       1.00      0.88      0.93         8
           4       1.00      0.60      0.75        15
           5       0.91      0.91      0.91        11
           6       0.93      1.00      0.96       532
           7       0.96      0.78      0.86        32
           8       1.00      0.79      0.88        19
           9       1.00      0.82      0.90        17
          10       1.00      0.90      0.95        10
          11       1.00      0.82      0.90        11
          12       1.00      0.62      0.76        26
          13       1.00      0.80      0.89        10

    accuracy                           0.94       750
   macro avg       0.99      0.82      0.89       750
weighted avg       0.94      0.94      0.94       750



## Label-2 calculation

In [64]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

df_2=df.copy()
df_2=df_2.dropna()
valid_2=valid.copy()
valid_2=valid_2.dropna()
X_train_2=df_2.drop(columns=['label_1','label_2','label_3','label_4'])
y_train_2=df_2.label_2
X_valid_2=valid_2.drop(columns=['label_1','label_2','label_3','label_4'])
y_valid_2=valid_2.label_2
X_test_2 = test.copy().drop(columns=['ID'])



X_2=df_2.drop(columns=['label_1','label_2','label_3','label_4'])
y_2=df_2.label_2


# knn_2=KNeighborsClassifier()
# cross_val_score(knn_2, X_2, y_2, cv=5, scoring='f1_macro').mean()


In [65]:
discrete_features = X_train_2.dtypes == int

mi_scores = make_mi_scores(X_train_2, y_train_2, discrete_features)
mi_scores.sort_values(ascending=False)

In [66]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

In [67]:
X_train_2 = X_train_2.drop(columns=last_100_features)
X_valid_2 = X_valid_2.drop(columns=last_100_features)
X_test_2 = X_test_2.drop(columns=last_100_features)

In [68]:
model_2=SVC()
model_2.fit(X_train_2, y_train_2)

SVC()

In [69]:
y_pred_2 = model_2.predict(X_valid_2)
y_test_2= model_2.predict(X_test_2)

In [70]:
print(classification_report(y_valid_2, y_pred_2))

              precision    recall  f1-score   support

        22.0       0.89      0.89      0.89        36
        23.0       0.85      0.89      0.87        71
        24.0       0.93      0.93      0.93        46
        25.0       0.92      0.76      0.83        79
        26.0       0.69      0.96      0.80       115
        27.0       0.97      0.84      0.90        81
        28.0       0.89      0.85      0.87        46
        29.0       1.00      0.89      0.94        45
        30.0       0.96      0.92      0.94        48
        31.0       0.89      0.86      0.88        65
        32.0       1.00      0.73      0.84        11
        33.0       1.00      0.87      0.93        30
        34.0       0.92      1.00      0.96        11
        35.0       1.00      0.91      0.95        11
        36.0       1.00      1.00      1.00         8
        41.0       1.00      0.71      0.83        14
        61.0       1.00      0.95      0.97        19

    accuracy              

## Label-3 calculation

In [71]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.utils import resample

df_3=df.copy()
valid_3=valid.copy()
X_train_3=df_3.drop(columns=['label_1','label_2','label_3','label_4'])
y_train_3=df_3.label_3
X_valid_3=valid_3.drop(columns=['label_1','label_2','label_3','label_4'])
y_valid_3=valid_3.label_3
X_test_3 = test.copy().drop(columns=['ID'])

# knn_3=KNeighborsClassifier()
# cross_val_score(knn_3, X_3, y_3, cv=5, scoring='f1_macro').mean()


In [72]:
discrete_features = X_train_3.dtypes == int

mi_scores = make_mi_scores(X_train_3, y_train_3, discrete_features)
mi_scores.sort_values(ascending=False)

In [73]:
# Get the last 100 feature names
last_100_features = mi_scores.index[-100:]

# Print the last 100 feature names
print(last_100_features)

In [74]:
model_3= SVC()

df_3=df.copy()
df_class_0 = df_3[df_3['label_3'] == 0]
df_class_1 = df_3[df_3['label_3'] == 1]

# Undersample the majority class (class 1)
undersampled_df_class_1 = resample(df_class_1,
                                   replace=False,  # Set to True if you want to sample with replacement
                                   n_samples=len(df_class_0),  # Match the number of samples in class 0
                                   random_state=42)  # Set a random seed for reproducibility

# Combine minority and down-sampled majority classes
df_balanced = pd.concat([df_class_0, undersampled_df_class_1])

# Shuffle the balanced dataset
df_3 = df_balanced.sample(frac=1, random_state=42)

X_train_3=df_3.drop(columns=['label_1','label_2','label_3','label_4'])
y_train_3=df_3.label_3
X_test_3 = test.copy().drop(columns=['ID'])

X_valid_3=valid_3.drop(columns=['label_1','label_2','label_3','label_4'])
y_valid_3=valid_3.label_3

# X_train_3 = X_train_3.drop(columns=last_100_features)
# X_valid_3 = X_valid_3.drop(columns=last_100_features)
# X_test_3 = X_test_3.drop(columns=last_100_features)



In [75]:
model_3.fit(X_train_3, y_train_3)

SVC()

In [76]:
y_pred_3 = model_3.predict(X_valid_3)
y_test_3= model_3.predict(X_test_3)

In [77]:
print(classification_report(y_valid_1, y_pred_1))

              precision    recall  f1-score   support

           1       1.00      0.92      0.96        13
           2       1.00      1.00      1.00         9
           3       0.85      0.92      0.88        12
           4       0.67      1.00      0.80        16
           5       1.00      0.83      0.91        18
           6       1.00      1.00      1.00         9
           7       0.84      0.94      0.89        17
           8       1.00      0.71      0.83        14
           9       1.00      0.91      0.95        11
          10       1.00      1.00      1.00         8
          11       0.79      1.00      0.88        19
          12       1.00      1.00      1.00         7
          13       1.00      0.73      0.84        11
          14       1.00      0.93      0.97        15
          15       1.00      0.88      0.94        17
          16       1.00      0.93      0.96        14
          17       1.00      0.93      0.96        14
          18       1.00    

## **final output**

In [78]:
new_df = pd.DataFrame({"ID": test["ID"], 'label_1': y_test_1 ,'label_2': y_test_2 ,'label_3': y_test_3 ,'label_4': y_test_4 })


In [79]:
new_df

,ID,label_1,label_2,label_3,label_4
0,1,26,22.0,0,2
1,2,18,25.0,1,6
2,3,16,30.0,1,6
3,4,7,27.0,1,6
4,5,58,29.0,0,6
...,...,...,...,...,...
739,740,11,26.0,1,6
740,741,35,24.0,1,2
741,742,54,27.0,1,6
742,743,38,32.0,1,12


In [80]:
new_df.to_csv('ML_project_layer_9.csv',index=False)